https://www.pinecone.io/learn/series/faiss/locality-sensitive-hashing/ (ref)

In [9]:
a = "flying fish flew by the space station"
b = "we will not allow you to bring your pet armadillo along"
c = "he figured a few sticks of dynamite were easier than a fishing pole to catch fish"

In [10]:
def shingle(text, k):
    shingle_set = []
    for i in range(len(text) - k+1):
        shingle_set.append(text[i:i+k])
    return set(shingle_set)

In [11]:
a_s = shingle(a, 2)
b_s = shingle(b, 2)
c_s = shingle(c, 2)

In [17]:
vocab = a_s | b_s | c_s
vocab_lst = list(vocab)
stoi = {s: i for i,s in enumerate(vocab_lst)}

In [16]:
len(vocab)

102

In [22]:
hash_ex = list(range(1, len(vocab)+1))
hash_ex[:2]

[1, 2]

In [23]:
hash_ex = list(range(1, len(vocab)+1))
print(hash_ex)  # we haven't shuffled yet

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102]


In [24]:
from random import shuffle

In [25]:
shuffle(hash_ex)

In [42]:
import numpy as np

a_1hot = np.zeros(len(vocab), dtype=int)
b_1hot = np.zeros(len(vocab), dtype=int)
c_1hot = np.zeros(len(vocab), dtype=int)

for s in a_s:
    a_1hot[stoi[s]] = 1
for s in b_s:
    b_1hot[stoi[s]] = 1
for s in c_s:
    c_1hot[stoi[s]] = 1

In [44]:
for i in range(1, len(vocab)+1):
    idx = hash_ex.index(i)
    sig_val = b_1hot[idx]
    print(f"{i} -> {idx} -> {sig_val}")
    if sig_val==1:
        print('match!')
        break

1 -> 19 -> 0
2 -> 48 -> 0
3 -> 61 -> 1
match!


In [48]:
def create_hash_func(size):
    hash_ex = list(range(1, size))
    shuffle(hash_ex)
    return hash_ex

def build_minhash_func(vocab_size, nbits):
    hashes = []
    for _ in range(nbits):
        hashes.append(create_hash_func(vocab_size+1))
    return hashes

minhash_func = build_minhash_func(len(vocab), 20)
np.array(minhash_func).shape

(20, 102)

In [50]:
def create_hash(vector):
    signature = []
    for func in minhash_func:
        for i in range(1, len(vocab)+1):
            idx = func.index(i)
            sign_val = vector[idx]
            if sign_val == 1:
                signature.append(idx)
                break
    return signature

In [53]:
a_sig = create_hash(a_1hot)
b_sig = create_hash(b_1hot)
c_sig = create_hash(c_1hot)

In [60]:
def jaccard(a:set, b:set):
    return len(a.intersection(b))/len(a.union(b))

In [61]:
jaccard(a_s, b_s),jaccard(set(a_sig), set(b_sig))

(0.125, 0.10714285714285714)

In [65]:
jaccard(a_s, c_s), jaccard(set(a_sig), set(c_sig))

(0.24358974358974358, 0.13333333333333333)

In [63]:
jaccard(b_s, c_s), jaccard(set(b_sig), set(c_sig))

(0.15384615384615385, 0.12121212121212122)